In [1]:
import datetime
import nltk
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import pandas as pd
import numpy as np
import time
import math
import sys
import gensim
import os
import warnings
import torch
import itertools
import multiprocessing as mp
from collections import Counter, defaultdict
from inspect import signature
from scipy.stats import ks_2samp, hypergeom
from sklearn.metrics import precision_recall_curve, f1_score, auc
from sklearn.model_selection import train_test_split, KFold
from scipy import spatial, stats
from statsmodels.sandbox.stats.multicomp import multipletests
from nltk.corpus import brown
from nltk.tokenize import word_tokenize, sent_tokenize
from sklearn.neighbors import KNeighborsClassifier
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM
from gensim.parsing.preprocessing import strip_non_alphanum, stem_text, preprocess_string, remove_stopwords
from gensim.utils import simple_preprocess
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import NMF
from sklearn.decomposition import LatentDirichletAllocation as LDA
from sklearn.cluster import AgglomerativeClustering

sys.path.append("../../oats")
from oats.utils.utils import save_to_pickle, load_from_pickle, merge_list_dicts, flatten, to_hms
from oats.utils.utils import function_wrapper_with_duration
from oats.biology.dataset import Dataset
from oats.biology.groupings import Groupings
from oats.biology.relationships import ProteinInteractions, AnyInteractions
from oats.annotation.ontology import Ontology
from oats.annotation.annotation import annotate_using_noble_coder
from oats.distances import pairwise as pw
from oats.distances.edgelists import merge_edgelists, make_undirected, remove_self_loops, subset_with_ids

mpl.rcParams["figure.dpi"] = 400
warnings.simplefilter('ignore')
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
nltk.download('punkt', quiet=True)
nltk.download('brown', quiet=True)
nltk.download('averaged_perceptron_tagger', quiet=True)

True

In [6]:
df = pd.read_csv("../outputs/04_25_2020_h18m07s00-phe-c/part_5_topic_modeling.csv")
df

,order,topic,GAM,GEM,EMG,MGD,EMB,MSD,NHM,SRL,GER,NLS,PIG,GRS,ROT,LEF,IST,ARC,MSL,FSM,OVP,SRF,SSC,FLT,SEN,CDR,MTM,STT,RTH,TCM,CUL,PRA,CPR,WAT,TMP,LIT,MEC,MPH,NUT,HRM,CHS,MCH,PTH,OBI
0,0,2,0.201135,0.170821,0.105864,0.131966,0.003965,0.062218,0.001990,0.005818,0.012997,0.008160,0.003360,0.007222,0.009188,0.006382,0.007466,0.008573,0.006981,0.014048,0.015899,0.025470,0.002849,0.003653,0.006477,0.000109,0.008312,0.003316,0.007008,0.011741,0.006776,0.001291,0.003982,0.001959,0.003374,0.002993,0.002719,0.009549,0.003437,0.001103,0.003591,0.000145,0.001386,0.000000
1,1,0,0.014102,0.043929,0.047568,0.006710,0.050125,0.031316,0.000929,0.012006,0.004321,0.001454,0.007854,0.002182,0.002630,0.002702,0.002286,0.000655,0.002637,0.001908,0.001312,0.002108,0.002660,0.001696,0.000257,0.000244,0.000247,0.000310,0.001056,0.003251,0.006704,0.000910,0.001680,0.000322,0.002036,0.001044,0.000000,0.001704,0.001129,0.001385,0.001218,0.000631,0.000771,0.006522
2,2,23,0.010952,0.009552,0.029117,0.011772,0.051172,0.036892,0.002735,0.017068,0.009457,0.006863,0.010339,0.005820,0.006067,0.006051,0.005817,0.006100,0.005884,0.011471,0.013848,0.009346,0.004295,0.004697,0.002813,0.000914,0.001126,0.006198,0.007785,0.010383,0.006294,0.006084,0.001695,0.002877,0.008106,0.004624,0.001712,0.004358,0.005481,0.004584,0.004585,0.006800,0.001870,0.027941
3,3,17,0.003488,0.000782,0.008847,0.000824,0.051778,0.006724,0.000165,0.002013,0.001343,0.001009,0.002723,0.000420,0.000403,0.000715,0.000376,0.000386,0.003661,0.001036,0.002603,0.001167,0.000665,0.000333,0.000156,0.000064,0.000088,0.000064,0.000389,0.001061,0.000335,0.002233,0.000063,0.000402,0.003342,0.000171,0.000020,0.000906,0.001182,0.000833,0.000717,0.001160,0.000206,0.000558
4,4,9,0.001691,0.001469,0.007460,0.005341,0.044881,0.002752,0.005685,0.025017,0.010002,0.037447,0.016479,0.010986,0.010549,0.015262,0.015245,0.014277,0.012548,0.015025,0.004468,0.008689,0.008214,0.008167,0.011459,0.004397,0.003476,0.005775,0.007683,0.029239,0.006045,0.001441,0.000067,0.000951,0.006323,0.014914,0.028925,0.010129,0.006615,0.006594,0.002914,0.002429,0.000689,0.023171
5,5,14,0.000270,0.000252,0.001888,0.000205,0.020375,0.002149,0.000000,0.000221,0.000051,0.000225,0.000027,0.000267,0.000023,0.000289,0.000029,0.000043,0.000038,0.000498,0.000210,0.000386,0.000094,0.000478,0.000017,0.000047,0.000016,0.000082,0.000036,0.000322,0.007637,0.000559,0.000116,0.000006,0.000793,0.000029,0.000000,0.000000,0.000047,0.000475,0.000026,0.000001,0.000488,0.000000
6,6,29,0.015960,0.012433,0.009841,0.011504,0.007366,0.023242,0.283750,0.017253,0.008354,0.012490,0.012050,0.011784,0.008632,0.009780,0.008199,0.016835,0.019140,0.010733,0.008629,0.008347,0.009319,0.005756,0.007716,0.005196,0.006624,0.006491,0.006591,0.010227,0.011571,0.004321,0.007488,0.007156,0.010158,0.007391,0.006088,0.007049,0.004770,0.005920,0.005031,0.006829,0.004643,0.002815
7,7,7,0.004183,0.009735,0.009764,0.005280,0.009870,0.013138,0.009767,0.068657,0.015479,0.012301,0.029867,0.006454,0.007234,0.006290,0.004374,0.006092,0.005256,0.003494,0.000481,0.003498,0.002050,0.004488,0.005679,0.007864,0.002211,0.004049,0.003721,0.007868,0.005126,0.001820,0.001126,0.003365,0.008511,0.009328,0.013259,0.008421,0.007880,0.006798,0.004688,0.005667,0.002476,0.000000
8,8,18,0.007134,0.022958,0.024583,0.000683,0.001398,0.000000,0.000727,0.030593,0.000458,0.000853,0.027624,0.000902,0.000713,0.001104,0.000892,0.001263,0.000998,0.001159,0.003216,0.003064,0.017480,0.000770,0.001217,0.000444,0.000000,0.001048,0.001279,0.001599,0.000692,0.000826,0.000799,0.000372,0.001173,0.001049,0.001176,0.000898,0.000818,0.000429,0.001283,0.000654,0.000268,0.000000
9,9,22,0.001843,0.001303,0.007951,0.001803,0.010399,0.002131,0.014959,0.029106,0.020934,0.008469,0.070718,0.028112,0.007090,0.013119,0.013506,0.009046,0.021476,0.009081,0.003179,0.009764,0.017890,0.011160,0.007522,0.007048,0.000000,0.009390,0.004478,0.005970,0.009182,0.004101,0.002973,0.005413,0.007716,0.009731,0.025809,0.007931,0.007990,0.003821,0.007730,

In [8]:
o = Ontology("../ontologies/go.obo")
print("done")

done


In [7]:
len(o)


1941

In [ ]:
import pronto
o = pronto.Ontology("../ontologies/go-basic.obo")
print("done")

In [ ]:
# how fast is it to get root terms?
a = []
for t in o.terms():
    parents = [t for t in t.superclasses(with_self=False)]
    if t.name is not None:
        if len(parents) == 0:
            if "obsolete" not in t.name:
                a.append(t.id)
print(a)

In [ ]:
num_terms = len([t for t in o.terms()])
depths = {i:0 for i in a}
#checked = set(depths.keys())

depth = 1
cont = True
while cont:
    before = len(depths)
    print(len(depths))
    print(num_terms)
    
    new_terms = []
    for t in depths.keys():
        new_terms.extend([y.id for y in o[t].subclasses(with_self=False, distance=1)])
    
    for i in new_terms:
        if i not in depths:
            depths[i] = depth
    
    depth = depth + 1
    after = len(depths)
    if before == after:
        cont = False
    
print("done")

# this gets all the ones we can find using this method,
# and then we'll need to make sure to default to 1 on all the other terms (things that were excluded from the 
# previous hunt for whatever reason... no .name? obsolete? etc.)

In [ ]:
import pronto
pronto_ontology_obj = pronto.Ontology("../ontologies/go-basic.obo")


def get_term_depth_dictionary(pronto_ontology_obj):

    # Find the root term(s) of the ontology.
    root_term_ids = []
    for term in pronto_ontology_obj.terms():
        # Check if this term has no inherited terms (is a root), discounting terms that are obsolete.
        inherited_terms = [t for t in term.superclasses(with_self=False)]
        if (len(inherited_terms)==0) and (term.name is not None) and ("obsolete" not in term.name):
            root_term_ids.append(term.id)
            
    # Find the depths of all terms in the ontology below those terms.
    depths = {i:0 for i in root_term_ids}
    depth = 1
    done = False
    while not done:
        
        # Add all the terms immediately below 
        before = len(depths)
        new_terms = []
        for old_term_id in [i for i in depths.keys() if depths[i] == depth-1]:
            for new_term_id in [t.id for t in pronto_ontology_obj[old_term_id].subclasses(with_self=False,distance=1)]:
                if new_term_id not in depths:
                    depths[new_term_id] = depth
        
        # Increment the depth and see if any new terms were added to the distance dictionary during this pass.
        depth = depth + 1
        after = len(depths)
        if before == after:
            done = True
            
    # Add any other remaining terms to the dictionary with a depth of 0 indicating minimal specificity.
    for term in pronto_ontology_obj.terms():
        if term.id not in depths:
            depths[term.id] = 0
    
    # Return the dictionary mapping term IDs to their depth in the hierarchy.
    return(depths)
    
d = get_term_depth_dictionary(pronto_ontology_obj)

print(d)
print("done")

In [ ]:
terms = [term for term in o.terms()]


t = terms[100]
print(t.name)
print(t.id)


In [ ]:
a = [x.name for x in t.superclasses(with_self=False)]
print(a)

In [ ]:
o = Ontology("../../oats/tests/data/test_ontology.obo")
for k,v in o.subclass_dict.items():
    print(k,v)

In [ ]:
import pronto
pronto_o = pronto.Ontology("../ontologies/go-basic.obo")
print("done")
for t in pronto_o.terms():
    print(t.name)
    if (t.name is not None) and ("obsolete" not in t.name):    
        print(t.name)
        #pass
    else:
        print("HERE")

#import pronto
#pronto_o = pronto.Ontology("../ontologies/po.obo")

#term = pronto_o["PO:0000003"]

#for term in pronto_o.terms():
#sup = [t for t in term.subclasses(distance=1)]
#print(sup)
#break

In [ ]:
for t in pronto_o.terms():
    for s in t.synonyms:
        print(s.description)